In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

import numpy as np
import zipfile
import json
import pickle

from sklearn.model_selection import GridSearchCV

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Перебор параметров для подбора наилучших

In [2]:
f = open('dev-dataset-task2023-04.json')
raw_data = json.load(f)

X, y = [], []
for data, label in raw_data:
    X.append(data)
    y.append(label)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [3]:
from nltk.corpus import stopwords

tfidf = TfidfVectorizer(min_df=3, stop_words=stopwords.words('russian'))
X_tfidf_train = tfidf.fit_transform(X_train)

In [4]:
for n_neighbors in range(1, 5):
    for metrics in ['euclidean', 'cosine']:
        for weights in ['uniform', 'distance']:
            clf = KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights, metric=metrics)
            clf.fit(X_tfidf_train, y_train)
            preds = 0
            for i in range(len(X_test)):
                data1 = tfidf.transform([X_test[i]])
                pred = clf.predict(data1)
                if pred == y_test[i]:
                        preds += 1
            print(n_neighbors, metrics, weights, preds, len(y_test))

1 euclidean uniform 189 323
1 euclidean distance 189 323
1 cosine uniform 283 323
1 cosine distance 283 323
2 euclidean uniform 118 323
2 euclidean distance 189 323
2 cosine uniform 271 323
2 cosine distance 282 323
3 euclidean uniform 121 323
3 euclidean distance 189 323
3 cosine uniform 269 323
3 cosine distance 281 323
4 euclidean uniform 110 323
4 euclidean distance 189 323
4 cosine uniform 270 323
4 cosine distance 278 323


Получилось, что лучшие параметры 1 cosine uniform 283 323, либо 1 cosine distance 283 323, буду использовать uniform

In [2]:
f = open('dev-dataset-task2023-04.json')
raw_data = json.load(f)

In [3]:
X, y = [], []
for data, label in raw_data:
    X.append(data)
    y.append(label)

In [4]:
len(X)

3223

In [5]:
X = np.array(X)
y = np.array(y)

In [6]:
from nltk.corpus import stopwords

tfidf = TfidfVectorizer(min_df=3, stop_words=stopwords.words('russian'))

In [7]:
X_tfidf = tfidf.fit_transform(X)

In [8]:
clf = KNeighborsClassifier(n_neighbors=1, weights='uniform', metric ='cosine')
clf.fit(X_tfidf, y)

KNeighborsClassifier(metric='cosine', n_neighbors=1)

In [9]:
saved_file = "model.pkl"
with open(saved_file, "wb") as file:
    pickle.dump(clf, file)

In [10]:
saved_vectorizer = "transformer.pkl"
with open(saved_vectorizer, "wb") as file:
    pickle.dump(tfidf, file)

проверка на самой же выборке

In [11]:
from solution import Solution

In [12]:
cls = Solution()

In [14]:
preds = 0
for data, label in raw_data:
    pred = cls.predict(data)
    if pred == label:
        preds += 1

In [15]:
preds

3129

In [16]:
len(y)

3223